## Erstmal auf den Zyklus gruppieren und dann Features erstellen

### Import

In [1]:
import pandas as pd
import numpy as np

import os
import sys
import logging

import functions as fu
from feature_engineering import *

# Pfade setzen
c_fold = fu.get_folder()
p_fold = c_fold + '\data\Prepared'
folder = c_fold + '\data\Processed\Zyklus'


# Initialize Logging
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(
                            filename='datalogger.log',
                            mode='a',
                            encoding='utf-8'),
                        logging.StreamHandler(sys.stdout)])

In [38]:
def stark_leicht(type_, current_mean):
    if type_ == 'D': #discharge
        if current_mean >= 3:
            return "entladen stark"
        else:
            return "entladen leicht"
    if type_ == 'C': #charge
        if current_mean <= -3:
            return "laden stark"
        else:
            return "laden leicht"
    if type_ == 'R':
        return "pause"

    

def temp_hoch(temperature_amax):
    if temperature_amax >= 30:
        return "warm"
    else:
        return "nicht warm"
        

# for file in os.listdir(p_fold):
#     if ".csv" in file:
file = "prepared_RW1.csv"
logging.info(f"<<< Started processing file: {file} >>>")
new_filename = file.replace("prepared","processed")

df = pd.read_csv(f"{p_fold}\\{file}", index_col = 0,
                         #nrows= 100000
                        )
logging.info(f"<<< Size of Dataframe: {df.shape} >>>")

#Auf Ebene Batterie + Zyklus gruppieren mit benötigten Aggregationen
df_pv = pd.pivot_table(
   df,
   index=['batteryname','zyklus','comment','type'],
   aggfunc={'time': np.min, 'relativeTime':  np.max, 'voltage': [np.min, np.max, np.mean], 'current': [np.min, np.max, np.mean], 'temperature': [np.min, np.max, np.mean]}
).reset_index()

df_pv.columns = ['_'.join(col) for col in list(df_pv.columns)]



df_pv['lade_klassen'] = df_pv.apply(lambda x: stark_leicht(x['type_'],x['current_mean']),axis=1)
#Lade

df_pv['temp_klassen'] = df_pv.apply(lambda x: temp_hoch(x['temperature_amax']),axis=1)


logging.info(f"<<< Start saving dataframe with shape {df_pv.shape} >>>")
df_pv.to_csv(f"{folder}\\{new_filename}", sep=',', index=False)
logging.info(f"<<< Saved processed file in directory {folder}\\{new_filename} >>>")

[08:40:03] INFO: <<< Started processing file: prepared_RW1.csv >>>
[08:40:07] INFO: <<< Size of Dataframe: (2958840, 11) >>>
[08:40:08] INFO: <<< Start saving dataframe with shape (17065, 17) >>>
[08:40:09] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW1.csv >>>


In [32]:
df = pd.read_csv(f"{folder}\\{new_filename}")

In [35]:
df["ladearten"].value_counts()

pause              9085
entladen leicht    5003
entladen stark     1696
laden leicht       1281
Name: ladearten, dtype: int64